# Fully Connected Genomic Model (Only Geneic SNPs) 



In [1]:
import numpy  as np
import pandas as pd

import torch
from   torch import nn
import torch.nn.functional as F
from   torch.utils.data import Dataset
from   torch.utils.data import DataLoader

import lightning.pytorch as pl
from   lightning.pytorch.loggers import CSVLogger

from dataG2F.core import get_data
from dataG2F.qol  import ensure_dir_path_exists

from EnvDL.dlfn import BigDataset, plDNN_general
from EnvDL.sets import mask_parents # for creating sets on the fly

In [2]:
from EnvDL.dlfn import Linear_res_block

In [3]:
cache_path = '../nbs_artifacts/aim_1a_G_Gene_FCNr/'

In [4]:
# Run settings: 
params_run = {
    'batch_size': 256,
    'max_epoch' : 64,   
}

# data settings
params_data = {
    'y_var': 'Yield_Mg_ha',
    'y_resid': 'None', # None, Env, Geno
    'y_resid_strat': 'None', # None, naive_mean, filter_mean, ...
    'holdout_parents': [
        ## 2022 ====
        'LH244',
        ## 2021 ====
        'PHZ51',
        # 'PHP02',
        # 'PHK76',
        ## 2019 ====
        # 'PHT69',
        'LH195',
        ## 2017 ====
        # 'PHW52',
        # 'PHN82',
        ## 2016 ====
        # 'DK3IIH6',
        ## 2015 ====
        # 'PHB47',
        # 'LH82',
        ## 2014 ====
        # 'LH198',
        # 'LH185',
        # 'PB80',
        # 'CG102',
 ],    
}

# in this file I define params later. I've included it here to gurantee that we can merge other params dicts into it.
params = {
}

In [5]:
batch_size = params_run['batch_size']
max_epoch  = params_run['max_epoch']

y_var = params_data['y_var']

In [6]:

save_prefix = [e for e in cache_path.split('/') if e != ''][-1]

if 'None' != params_data['y_resid_strat']:
    save_prefix = save_prefix+'_'+params_data['y_resid_strat']

ensure_dir_path_exists(dir_path = cache_path)

In [7]:
use_gpu_num = 0
device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


## Load Data

In [8]:
obs_geno_lookup          = get_data('obs_geno_lookup')
phno                     = get_data('phno')
acgt                     = get_data('KEGG_slices')
acgt                     = np.concatenate(acgt, axis= 2)

In [9]:
# flatten data
acgt = acgt.reshape(4926, -1)

In [10]:
# make holdout sets
holdout_parents = params_data['holdout_parents']

# create a mask for parent genotype
mask = mask_parents(df= phno, col_name= 'Hybrid', holdout_parents= holdout_parents)

train_mask = mask.sum(axis=1) == 0
test_mask  = mask.sum(axis=1) > 0

train_idx = train_mask.loc[train_mask].index
test_idx  = test_mask.loc[test_mask].index

In [11]:
# convert y to residual if needed

if params_data['y_resid'] == 'None':
    pass
else:
    if params_data['y_resid_strat'] == 'naive_mean':
        # use only data in the training set (especially since testers will be more likely to be found across envs)
        # get enviromental means, subtract from observed value
        tmp = phno.loc[train_idx, ]
        env_mean = tmp.groupby(['Env_Idx']
                     ).agg(Env_Mean = (y_var, 'mean')
                     ).reset_index()
        tmp = phno.merge(env_mean)
        tmp.loc[:, y_var] = tmp.loc[:, y_var] - tmp.loc[:, 'Env_Mean']
        phno = tmp.drop(columns='Env_Mean')

    if params_data['y_resid_strat'] == 'filter_mean':
        # for adjusting to environment we could use _all_ observations but ideally we will use the same set of genotypes across all observations
        def minimum_hybrids_for_env(tmp = phno.loc[:, ['Env', 'Year', 'Hybrid']],
                                    year = 2014):
            # Within each year what hybrids are most common?
            tmp = tmp.loc[(tmp.Year == year), ].groupby(['Env', 'Hybrid']).count().reset_index().sort_values('Year')

            all_envs = set(tmp.Env)
            # if we filter on the number of sites a hybrid is planted at, what is the largest number of sites we can ask for before we lose a location?
            # site counts for sets which contain all envs
            i = max([i for i in list(set(tmp.Year)) if len(set(tmp.loc[(tmp.Year >= i), 'Env'])) == len(all_envs)])

            before = len(set(tmp.loc[:, 'Hybrid']))
            after  = len(set(tmp.loc[(tmp.Year >= i), 'Hybrid']))
            print(f'Reducing {year} hybrids from {before} to {after} ({round(100*after/before)}%).')
            tmp = tmp.loc[(tmp.Year >= i), ['Env', 'Hybrid']].reset_index(drop=True)
            return tmp


        tmp = phno.loc[:, ['Env', 'Year', 'Hybrid']]
        filter_hybrids = [minimum_hybrids_for_env(tmp = phno.loc[:, ['Env', 'Year', 'Hybrid']], year = i) 
                          for i in list(set(phno.Year)) ]
        env_mean = pd.concat(filter_hybrids).merge(phno, how = 'left')

        env_mean = env_mean.groupby(['Env_Idx']
                          ).agg(Env_Mean = (y_var, 'mean')
                          ).reset_index()

        tmp = phno.merge(env_mean)
        tmp.loc[:, y_var] = tmp.loc[:, y_var] - tmp.loc[:, 'Env_Mean']
        phno = tmp.drop(columns='Env_Mean')
        

In [12]:
# center and y value data
assert 0 == phno.loc[:, y_var].isna().sum()

y = phno.loc[:, y_var]
# use train index to prevent information leakage
y_c = y[train_idx].mean()
y_s = y[train_idx].std()

y = (y - y_c)/y_s

In [13]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs  = torch.from_numpy(np.array(train_idx)), 
    lookup_geno = torch.from_numpy(obs_geno_lookup),
    y =           torch.from_numpy(y.to_numpy()).to(torch.float32)[:, None],
    G =           torch.from_numpy(acgt).to(torch.float32),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)

validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs  = torch.from_numpy(np.array(test_idx)), 
    lookup_geno = torch.from_numpy(obs_geno_lookup),
    y =           torch.from_numpy(y.to_numpy()).to(torch.float32)[:, None],
    G =           torch.from_numpy(acgt).to(torch.float32),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)

## Define Parameters of model

In [14]:
# convenince wrapper to fill in for R's seq or x:y notation
def linrange(start, stop):
    import numpy as np
    diff = start - stop
    res = np.linspace(start, stop, abs(diff)+1).astype(int)
    return res 

In [15]:
[2**i for i in linrange(11, 4)]

[2048, 1024, 512, 256, 128, 64, 32, 16]

In [16]:
# This one is designed to go from (3x125891) -> ~1258.91  -> ~125.891 -> ~12.5891 -> 1
layer_sizes = [1024, 128, 12]
layer_drops = [0.1 for e in layer_sizes]

num_layers = len(layer_sizes)

params = {
    'num_layers':num_layers,
    f"in_1_of_{num_layers}": (4 * 75419)
}

for i in range(num_layers):
    params[f"out_{ i + 1}_of_{num_layers}"] = layer_sizes[i]
    params[f"drop_{ i + 1}_of_{num_layers}"] = layer_drops[i]
        

In [17]:
params

{'num_layers': 3,
 'in_1_of_3': 301676,
 'out_1_of_3': 1024,
 'drop_1_of_3': 0.1,
 'out_2_of_3': 128,
 'drop_2_of_3': 0.1,
 'out_3_of_3': 12,
 'drop_3_of_3': 0.1}

In [18]:
[e.shape for e in next(iter(training_dataloader))]

[torch.Size([256, 1]), torch.Size([256, 301676])]

In [19]:
# A quirk of this is that to get only a single layer the length of the input tensor must be passed in. for 2+ I'll figure it out.
class NeuralNetwork(nn.Module):
    def __init__(self, parameterization):
        super(NeuralNetwork, self).__init__()            
        module_list = []

        max_layer = parameterization['num_layers']
        for i in range(max_layer):
            if i  == 0:
                name_in = f"in_{i+1}_of_{max_layer}"
            else:
                name_in = f"out_{i}_of_{max_layer}"
            name_out = f"out_{i+1}_of_{max_layer}"
            name_drop= f"drop_{i+1}_of_{max_layer}"

            if i == 0:
                module_list += [nn.Flatten()]
            

            module_list += [
                Linear_res_block(
                    in_size  = parameterization[name_in], 
                    out_size = parameterization[name_out], 
                    drop_pr  = parameterization[name_drop])]
            
            if (i+1) == max_layer:
                module_list += [nn.Linear(parameterization[name_out], 1)]
                
        self.x_network = nn.ModuleList(module_list)
        
    def forward(self, x):
        
        for mod in self.x_network:
            if mod == self.x_network[-1]:
                out = x # get the penultimate layer's outputs for later
            x = mod(x)
        
        pred = x
        return pred

In [20]:
model = NeuralNetwork(parameterization = params).to(device)

In [21]:
# model(next(iter(training_dataloader))[1])[0:5]

In [22]:
torch.set_float32_matmul_precision('medium')

DNN = plDNN_general(model)  

optimizer = DNN.configure_optimizers()

logger = CSVLogger("nifa_tb", name=save_prefix)
logger.log_hyperparams(params={
    'params': params,
    'params_data': params_data,
    'params_run' : params_run,
    'misc': {
        'n_train': len(train_idx),
        'n_test':  len(test_idx)
    }
})

trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNN, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)


/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | mod  | NeuralNetwork | 311 M 
---------------------------------------
311 M     Trainable params
0         Non-trainable params
311 M     Total params
1,244.730 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=64` reached.


In [23]:
# save model with versioning. 
# this is hacky but should work
import os
versions = os.listdir('nifa_tb/'+save_prefix)
versions.sort()
torch.save(DNN.mod, cache_path+versions[-1]+'.pth')